This file is to practice implementing quantum algorithms as I keep learning them

Deutsch's algorithm

In [1]:
from qiskit import QuantumCircuit

In [2]:
def deutsch_function(case: int):
    """
    Generate a valid Deutsch algorithm as a QuantumCircuit
    """
    if case not in [1,2,3,4]:
        raise ValueError("'case' must be 1, 2, 3, or 4.")
    
    qc = QuantumCircuit(2)
    if case in [2,3]:
        qc.cx(0,1)
    if case in [3,4]:
        qc.x(1)
    
    return qc

In [32]:
display(deutsch_function(4).draw())

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

In [4]:
def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in Deutsch's algorithm
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n+1,n)

    qc.x(n)
    qc.h(range(n+1))

    qc.barrier()
    qc.compose(function, inplace=True)
    qc.barrier()

    qc.h(range(n))
    qc.measure(range(n), range(n))

    return qc

In [29]:
display(compile_circuit(deutsch_function(4)).draw())

┌───┐      ░       ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘ ░       ║ 
c: 1/═══════════════════════════╩═
                                0

In [26]:
from qiskit_aer import AerSimulator

def deutsch_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch function is constant or balanced.
    """

    qc = compile_circuit(function)
    
    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if measurements[0] == "0":
        return "constant"
    return "balanced"

In [27]:
f = deutsch_function(3)
display(f.draw())
display(deutsch_algorithm(f))

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

'balanced'